In [1]:
import torch.nn
from pytube import YouTube
yt = YouTube('https://www.youtube.com/watch?v=1VhJDJedRpg')

In [2]:
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path='youtube_video/', filename='example1.mp4')

'/Users/khoavo2003/PycharmProjects/DAAD-RISE-Germany/youtube_video/example1.mp4'

In [1]:
from moviepy.editor import VideoFileClip

def get_video_duration(video_path):
  clip = VideoFileClip(video_path)
  return clip.duration

# Example usage
video_duration = get_video_duration("/Users/khoavo2003/PycharmProjects/DAAD-RISE-Germany/critical_classification/dashcam_video/original_video/Vollbremsungen Überholen durchs Bankett und Schleicherei  DDG Dashcam Germany  580.mp4")
print(f"Video duration: {video_duration} seconds")

Video duration: 500.32 seconds


In [2]:
for i in range(4):
  for j in range(4):
    if i == j:
      continue
    print(i, j)

0 1
0 2
0 3
1 0
1 2
1 3
2 0
2 1
2 3
3 0
3 1
3 2


In [ ]:
import cv2
import numpy as np

def draw_gaussian_base(image, center, width, height, sigma=1):
  """
  Draws a 2D Gaussian base on the given image with specified center, width, height, and sigma.

  Args:
      image: The input image (numpy array).
      center: A tuple (x, y) representing the center of the Gaussian base.
      width: The width (standard deviation) of the Gaussian base in the x-direction.
      height: The height (standard deviation) of the Gaussian base in the y-direction.
      sigma: (Optional) The overall scaling factor for the Gaussian (default 1).
  """
  rows, cols = image.shape[:2]

  # Calculate coordinates for the Gaussian mask
  x0, y0 = center
  x = np.arange(0, cols, 1, float) - x0
  y = np.arange(0, rows, 1, float) - y0
  x, y = np.meshgrid(x, y)

  # Define the Gaussian function
  mask = np.exp(-((x**2 / (2*width**2)) + (y**2 / (2*height**2)))) * sigma

  # Adjust mask values to range between 0 and 1
  mask = mask / np.max(mask)

  # Apply the mask to the image (assuming single channel image, adjust for BGR if needed)
  masked_image = image.copy() 
  masked_image[mask > 0.5] = image[mask > 0.5] * mask[mask > 0.5]

  return masked_image


In [ ]:
# heatmap_drawing_test.py

import numpy as np
import cv2
from termcolor import colored

# ref:
# https://stackoverflow.com/questions/76723027/how-to-draw-2d-gaussian-blob-on-an-opencv-image/76724003#76724003

HEATMAP_WIDTH = 1600
HEATMAP_HEIGHT = 1000

def main():
    # suppress numpy printing in scientific notation
    np.set_printoptions(suppress=True)

    # create blank heatmap (OpenCV image)
    heatmap = np.zeros((HEATMAP_HEIGHT, HEATMAP_WIDTH), dtype=np.uint8)

    blob_1_center_x = 1599
    blob_1_center_y = 0
    blob_1_width = 451
    blob_1_height = 201

    gaussian_blob_1 = make_gaussian_blob(blob_1_width, blob_1_height)

    blob_2_center_x = 1100
    blob_2_center_y = 500
    blob_2_width = 451
    blob_2_height = 201

    gaussian_blob_2 = make_gaussian_blob(blob_2_width, blob_2_height)

    heatmap = add_gaussian_blob_to_heatmap(gaussian_blob_1, blob_1_center_x, blob_1_center_y, heatmap)
    heatmap = add_gaussian_blob_to_heatmap(gaussian_blob_2, blob_2_center_x, blob_2_center_y, heatmap)

    # show the heatmap
    cv2.imshow('heatmap', heatmap)

    cv2.waitKey(0)
# end function

def make_gaussian_blob(blob_width, blob_height):
    assert blob_height % 2 == 1 and blob_width % 2 == 1, \
        colored('\n\n' + 'in make_gaussian_blob, blob_height and blob_width must be odd numbers !!' + '\n', color='red', attrs=['bold'])

    # Create a 2D Gaussian blob
    # +-2.5 was derived from experimentation
    x, y = np.meshgrid(np.linspace(-2.5, 2.5, blob_width), np.linspace(-2.5, 2.5, blob_height))
    
    gaussian_blob = np.exp(-0.5 * (x**2 + y**2))

    # scale up the gaussian blob from the 0.0 to 1.0 range to the 0 to 255 range
    gaussian_blob = gaussian_blob * 255.0
    gaussian_blob = np.clip(gaussian_blob, a_min=0.0, a_max=255.0)
    gaussian_blob = np.rint(gaussian_blob).astype(np.uint8)

    return gaussian_blob
# end function

def add_gaussian_blob_to_heatmap(gaussian_blob, blob_center_x, blob_center_y, heatmap):
    # ToDo: this function is not perfect, there is a slight seam when two blobs overlap each other, eventually should resolve this

    blob_height, blob_width = gaussian_blob.shape[0:2]
    blob_left_edge_loc = round(blob_center_x - ((blob_width - 1) * 0.5))
    blob_right_edge_loc = round(blob_center_x + ((blob_width - 1) * 0.5))
    gaussian_left_edge = 0
    gaussian_right_edge = gaussian_blob.shape[1] - 1
    if blob_left_edge_loc < 0:
        gaussian_left_edge = -blob_left_edge_loc
        blob_left_edge_loc = 0
    if blob_right_edge_loc >= heatmap.shape[1]:
        gaussian_right_edge = gaussian_blob.shape[1] - 1 - (blob_right_edge_loc - (heatmap.shape[1] - 1))
        blob_right_edge_loc = heatmap.shape[1] - 1
        
    blob_top_edge_loc = round(blob_center_y - ((blob_height - 1) * 0.5))
    blob_bottom_edge_loc = round(blob_center_y + ((blob_height - 1) * 0.5))
    gaussian_top_edge = 0
    gaussian_bottom_edge = gaussian_blob.shape[0] - 1
    if blob_top_edge_loc < 0:
        gaussian_top_edge = -blob_top_edge_loc
        blob_top_edge_loc = 0
    if blob_bottom_edge_loc >= heatmap.shape[0]:
        gaussian_bottom_edge = gaussian_blob.shape[0] - 1 - (blob_bottom_edge_loc - (heatmap.shape[0] - 1))
        blob_bottom_edge_loc = heatmap.shape[0] - 1

    heatmap = heatmap.astype(np.uint16)
    gaussian_blob = gaussian_blob.astype(np.uint16)

    heatmap[blob_top_edge_loc:blob_bottom_edge_loc, blob_left_edge_loc:blob_right_edge_loc] += gaussian_blob[gaussian_top_edge:gaussian_bottom_edge,gaussian_left_edge:gaussian_right_edge]

    heatmap = np.where(heatmap > 255, 255, heatmap).astype(np.uint8)

    return heatmap

main()

In [1]:
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification

class_labels = sorted({item for item in ['critical', 'non_critical']})
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

model_ckpt = "MCG-NJU/videomae-base"
image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
model = VideoMAEForVideoClassification.from_pretrained(
    model_ckpt,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

/Users/khoavo2003/anaconda3/envs/daad/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
type(model)

transformers.models.videomae.modeling_videomae.VideoMAEForVideoClassification

In [4]:
import torch
isinstance(model, torch.nn.Module)

True

In [6]:
import os
os.path.exists('/Users/khoavo2003/PycharmProjects/DAAD-RISE-Germany/critical_classification/dashcam_video/original_video/Vollbremsungen Überholen durchs Bankett und Schleicherei  DDG Dashcam Germany  580_mask.mp4')

True

In [7]:
os.path.exists('/Users/khoavo2003/PycharmProjects/DAAD-RISE-Germany/critical_classification/dashcam_video/original_video/Vollbremsungen Überholen durchs Bankett und Schleicherei  DDG Dashcam Germany  580.mp4')

True

In [1]:
import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from typing import Dict

/Users/khoavo2003/anaconda3/envs/daad/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/khoavo2003/anaconda3/envs/daad/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
# Device on which to run the model
# Set to cuda to load on GPU
device = "cpu"

# Pick a pretrained model and load the pretrained weights
model_name = "slowfast_r50"
model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)

# Set to eval mode and move to desired device
model = model.to(device)
model = model.eval()

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /Users/khoavo2003/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOWFAST_8x8_R50.pyth" to /Users/khoavo2003/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R50.pyth
100%|██████████| 264M/264M [00:13<00:00, 21.2MB/s] 


In [3]:
!wget https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json
with open("kinetics_classnames.json", "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

--2024-06-25 08:31:28--  https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 2600:9000:223e:a800:13:6e38:acc0:93a1, 2600:9000:223e:6200:13:6e38:acc0:93a1, 2600:9000:223e:5200:13:6e38:acc0:93a1, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|2600:9000:223e:a800:13:6e38:acc0:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10326 (10K) [text/plain]
Saving to: ‘kinetics_classnames.json’

kinetics_classnames 100%[===================>]  10.08K  --.-KB/s    in 0.003s  

2024-06-25 08:31:28 (3.60 MB/s) - ‘kinetics_classnames.json’ saved [10326/10326]



In [4]:
####################
# SlowFast transform
####################

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

In [5]:
# Download the example video file
!wget https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4
# Load the example video
video_path = "archery.mp4"

# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0
end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class
video = EncodedVideo.from_path(video_path)

# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
video_data = transform(video_data)

# Move the inputs to the desired device
inputs = video_data["video"]
inputs = [i.to(device)[None, ...] for i in inputs]

--2024-06-25 08:31:55--  https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 2600:9000:223e:a800:13:6e38:acc0:93a1, 2600:9000:223e:6200:13:6e38:acc0:93a1, 2600:9000:223e:5200:13:6e38:acc0:93a1, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|2600:9000:223e:a800:13:6e38:acc0:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549197 (536K) [video/mp4]
Saving to: ‘archery.mp4’

archery.mp4         100%[===================>] 536.33K  --.-KB/s    in 0.04s   

2024-06-25 08:31:56 (13.1 MB/s) - ‘archery.mp4’ saved [549197/549197]



In [16]:
video_data['video'][0].shape

torch.Size([3, 8, 256, 256])

In [29]:
# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

In [24]:
video_data['video'].shape

torch.Size([3, 64, 240, 320])

In [25]:
video_data = transform(video_data)

In [31]:
video_data['video'].shape

torch.Size([3, 64, 240, 320])

In [3]:
import cv2
import numpy as np

# Define the video properties
frame_width = 640
frame_height = 480
frame_rate = 30
duration = 5  # in seconds
num_frames = frame_rate * duration

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs like 'MJPG', 'X264', etc.
out = cv2.VideoWriter('random_video.mp4', fourcc, frame_rate, (frame_width, frame_height))

for _ in range(num_frames):
    # Create a random frame
    frame = np.random.randint(0, 256, (frame_height, frame_width, 3), dtype=np.uint8)
    
    # Write the frame into the video file
    out.write(frame)

# Release everything if job is finished
out.release()

print("Random video saved as 'random_video.avi'")


Random video saved as 'random_video.avi'


In [1]:
import tensorflow as tf
import tensorflow.keras as keras

@keras.utils.register_keras_serializable(package="MyLayers")
class BinaryModel(tf.keras.Model):
    def __init__(self,name=None):
        super(BinaryModel, self).__init__(name=name)
        # Define your RNN layer
        self.rnn_for_extract_feature_per_object = tf.keras.layers.LSTM(64, return_sequences=False)
        # Define the MLP layers
        self.global_average = tf.keras.layers.GlobalAveragePooling1D()
        self.dense1 = tf.keras.layers.Dense(16, activation='elu')
        self.dense2 = tf.keras.layers.Dense(4, activation='elu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')
        self.input_feature_shape = 1280
        
    # def build(self, input_shape):
    #     self.rnn_for_extract_feature_per_object = self.add_weight(
    #         shape=(None, None, self.input_feature_shape),
    #         initializer="random_normal",
    #         trainable=True,
    #     )
    #     self.dense1 = self.add_weight(
    #         shape=(None, 64),
    #         initializer="random_normal",
    #         trainable=True,
    #     )
    #     self.dense2 = self.add_weight(
    #         shape=(None, 16),
    #         initializer="random_normal",
    #         trainable=True,
    #     )
    #     self.output_layer = self.add_weight(
    #         shape=(None, 4),
    #         initializer="random_normal",
    #         trainable=True,
    #     )

    def call(self, features_list, training=None, mask=None):
        # In this implementation, input is a dict[int, tf.Tensor], shape (None, 1280)
        final_feature_per_object = []
        for _, features in features_list.items():
            final_feature_per_object.append(self.rnn_for_extract_feature_per_object(tf.expand_dims(features, axis=0)))
        if len(final_feature_per_object) == 1:
            final_feature_per_object_reshape = tf.expand_dims(final_feature_per_object[0], axis=0)
        else:
            final_feature_per_object_reshape = tf.stack(final_feature_per_object, axis=1)
        final_feature = self.global_average(final_feature_per_object_reshape)
        x = self.dense1(final_feature)
        x = self.dense2(x)
        return self.output_layer(x)

In [2]:
# Instantiate the model
model = BinaryModel()
# Call and Verify the save model
print(model.call({0: tf.zeros((1, 1280))}))
# model.summary()
# Saving the model
model.save_weights('binary_model', save_format='tf')

tf.Tensor([[0.5]], shape=(1, 1), dtype=float32)


In [3]:
# Instantiate the model
model_load = BinaryModel()
# Loading the model
model_load.load_weights('binary_model')

In [10]:
print(model_load.call({0: tf.zeros((1, 1280))}))

tf.Tensor([[0.5]], shape=(1, 1), dtype=float32)
